In [1]:
# Libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
import urllib
import urllib2
import re
import unicodedata
import datetime
import requests
import requests_cache
requests_cache.install_cache('demo_cache')
%matplotlib inline

## Helper functions

In [3]:
def get_date(gid):
    return datetime.date(int(gid[:4]), int(gid[4:6]), int(gid[6:8]))

In [4]:
def get_game_num(soup):
    game_info = soup.findAll('h2')
    game_info = [tag for tag in game_info if tag.text != '']
    s1 = re.findall(r"[\w']+", str(game_info[2].text))
    g_num_1 = int(s1[len(s1)-1]) + int(s1[len(s1)-2])
    s2 = re.findall(r"[\w']+", str(game_info[3].text))
    g_num_2 = int(s2[len(s2)-1]) + int(s2[len(s2)-2])
    return g_num_1, g_num_2

In [5]:
def get_teams(soup):
    teams = soup.findAll('a', href=re.compile("(teams)"))
    team1 = teams[1].get('href')[7:10]
    team2 = teams[2].get('href')[7:10]
    return team1, team2

In [444]:
def time_to_sec(time):
    try:
        time_list = time.split(':')
        return int(time_list[0]) * 60 + int(time_list[1])
    except ValueError:
        return 0

## Testing and debugging

In [401]:
link = 'https://www.basketball-reference.com/boxscores/198801300CLE.html'
#link = 'https://www.basketball-reference.com/boxscores/198310280NJN.html'
page = urllib.urlopen(link).read()
soup = BeautifulSoup(page, "lxml")

In [402]:
game_info = soup.findAll('h2')
game_info = [tag for tag in game_info if tag.text != '']
s1 = re.findall(r"[\w']+", str(game_info[2].text))
g_num_1 = int(s1[len(s1)-1]) + int(s1[len(s1)-2])
s2 = re.findall(r"[\w']+", str(game_info[3].text))
g_num_2 = int(s2[len(s2)-1]) + int(s2[len(s2)-2])
print g_num_1, g_num_2

40 42


In [403]:
g_num_1, g_num_2 = get_game_num(soup)
print g_num_1, g_num_2

40 42


In [404]:
gid[6:8]

'30'

In [405]:
gid[:4]

'1988'

In [406]:
dat = datetime.date(1984,6,5)
dat.month

6

In [407]:
get_date(gid)

datetime.date(1988, 1, 30)

In [408]:
teams = soup.findAll('a', href=re.compile("(teams)"))
teams[1].get('href')[7:10]

'WSB'

In [409]:
# get game num
game_info = soup.findAll('h2')
print game_info
s1 = re.findall(r"[\w']+", str(game_info[3].text))
g_num_1 = int(s1[len(s1)-1]) + int(s1[len(s1)-2])
s2 = re.findall(r"[\w']+", str(game_info[5].text))
print s2
g_num_2 = int(s2[len(s2)-1]) + int(s2[len(s2)-2])
g_num_1, g_num_2

[<h2></h2>, <h2>Line Score</h2>, <h2>Four Factors</h2>, <h2>Washington Bullets (17-23)</h2>, <h2></h2>, <h2>Cleveland Cavaliers (21-21)</h2>, <h2></h2>, <h2>Team and League Schedules</h2>, <h2>Full Site Menu</h2>, <h2>We're Social...for Statheads</h2>, <h2>FAQs, Tip &amp; Tricks</h2>]
['Cleveland', 'Cavaliers', '21', '21']


(40, 42)

In [410]:
get_game_num(soup)

(40, 42)

In [411]:
get_teams(soup)

('WSB', 'CLE')

In [412]:
table_basic = soup.find('table', id='box_cle_basic')

In [413]:
table_basic

<table class="sortable stats_table" data-cols-to-freeze="1" id="box_cle_basic"><caption>Cleveland Cavaliers (21-21) Table</caption>\n<colgroup><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col></colgroup>\n<thead>\n<tr class="over_header"><th></th>\n<th aria-label="" class=" over_header center" colspan="19" data-stat="header_tmp">Basic Box Score Stats</th>\n</tr>\n<tr>\n<th aria-label="Starters" class=" poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>\n<th aria-label="Minutes Played" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>\n<th aria-label="Field Goals" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>\n<th aria-label="Field Goal Attempts

In [414]:
table_adv = soup.find('table', id='box_cle_advanced')

In [415]:
table_adv

<table class="sortable stats_table" data-cols-to-freeze="1" id="box_cle_advanced"><caption> Table</caption>\n<colgroup><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col></colgroup>\n<thead>\n<tr class="over_header"><th></th>\n<th aria-label="" class=" over_header center" colspan="15" data-stat="header_tmp">Advanced Box Score Stats</th>\n</tr>\n<tr>\n<th aria-label="Starters" class=" poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>\n<th aria-label="Minutes Played" class=" poptip center" data-over-header="Advanced Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>\n<th aria-label="True Shooting Percentage" class=" poptip center" data-over-header="Advanced Box Score Stats" data-stat="ts_pct" data-tip="&lt;strong&gt;True Shooting Percentage&lt;/strong&gt;&lt;br&gt;A measure of shooting efficiency that takes into

In [416]:
rows1 = table_basic.findAll('tr')
rows2 = table_adv.findAll('tr')

In [417]:
len(rows1[12])

20

In [418]:
rows2[2]

<tr><th class="left " csk="Price,Mark" data-append-csv="pricema01" data-stat="player" scope="row"><a href="/players/p/pricema01.html">Mark Price</a></th><td class="right " csk="2640" data-stat="mp">44:00</td><td class="right " data-stat="ts_pct">.586</td><td class="right " data-stat="efg_pct">.525</td><td class="right " data-stat="fg3a_per_fga_pct">.150</td><td class="right " data-stat="fta_per_fga_pct">.250</td><td class="right " data-stat="orb_pct">5.7</td><td class="right " data-stat="drb_pct">9.3</td><td class="right " data-stat="trb_pct">7.4</td><td class="right " data-stat="ast_pct">27.8</td><td class="right " data-stat="stl_pct">1.1</td><td class="right " data-stat="blk_pct">1.4</td><td class="right " data-stat="tov_pct">8.3</td><td class="right " data-stat="usg_pct">23.4</td><td class="right " data-stat="off_rtg">131</td><td class="right " data-stat="def_rtg">120</td></tr>

In [419]:
cols = rows2[2].findAll('td')

In [420]:
cols

[<td class="right " csk="2640" data-stat="mp">44:00</td>,
 <td class="right " data-stat="ts_pct">.586</td>,
 <td class="right " data-stat="efg_pct">.525</td>,
 <td class="right " data-stat="fg3a_per_fga_pct">.150</td>,
 <td class="right " data-stat="fta_per_fga_pct">.250</td>,
 <td class="right " data-stat="orb_pct">5.7</td>,
 <td class="right " data-stat="drb_pct">9.3</td>,
 <td class="right " data-stat="trb_pct">7.4</td>,
 <td class="right " data-stat="ast_pct">27.8</td>,
 <td class="right " data-stat="stl_pct">1.1</td>,
 <td class="right " data-stat="blk_pct">1.4</td>,
 <td class="right " data-stat="tov_pct">8.3</td>,
 <td class="right " data-stat="usg_pct">23.4</td>,
 <td class="right " data-stat="off_rtg">131</td>,
 <td class="right " data-stat="def_rtg">120</td>]

In [421]:
cols[1].text

u'.586'

In [422]:
rows2[2].findAll('a')[0].text

u'Mark Price'

In [425]:
len(rows2[2]), len(rows2[3])

(16, 16)

In [427]:
info = rows1[5].findAll('th')
name = info[0].findAll('a')
name[0].get('href')[11:20]

'williho01'

In [428]:
rows2 = table_adv.findAll('tr')
rows = table_basic.findAll('tr')
rows = rows[:-1]
type(rows)

list

In [233]:
link = 'https://www.basketball-reference.com/boxscores/199501120DEN.html'
page = urllib.urlopen(link).read()
soup = BeautifulSoup(page, "lxml")
tables = soup.findAll('table', id=re.compile("(advanced|basic)"))

In [271]:
rows = tables[0].findAll('tr')
rows = rows[:-1]
game_log_basic = pd.DataFrame()
for row in rows:
    print len(row)
    if (len(row) == 21) | (len(row) == 20):
        tds = row.findAll('td')
        #print row.findAll('a')
        player = row.findAll('a')[0].text
        info = row.findAll('th')
        name = info[0].findAll('a')
        player_link = name[0].get('href')
        player_id = (player_link.split('/')[3])[:-5]
        game_log_basic = game_log_basic.append({'player': player,
                                    'link': player_link,
                                    'id': player_id,
                                    'mp': tds[0].text,
                                    'sec': int(time_to_sec(tds[0].text)),
                                    'fg': int(tds[1].text),
                                    'fga': int(tds[2].text),
                                    'fg_pct': tds[3].text,
                                    'fg3': int(tds[4].text),
                                    'fg3a': int(tds[5].text),
                                    'fg3_pct': tds[6].text,
                                    'ft': int(tds[7].text),
                                    'fta': int(tds[8].text),
                                    'ft_pct': tds[9].text,
                                    'orb': int(tds[10].text),
                                    'drb': int(tds[11].text),
                                    'trb': int(tds[12].text),
                                    'ast': int(tds[13].text),
                                    'stl': int(tds[14].text),
                                    'blk': int(tds[15].text),
                                    'tov': int(tds[16].text),
                                    'pf': int(tds[17].text),
                                    'pts': int(tds[18].text)},
                                    #'PM': int(tds[19].text)}, 
                                ignore_index=True)
        
rows = tables[1].findAll('tr')
rows = rows[:-1]
game_log_adv = pd.DataFrame()
for row in rows:
    print len(row)
    if len(row) == 16:
        tds = row.findAll('td')
        #print row.findAll('a')
        #player = row.findAll('a')[0].text
        #info = row.findAll('th')
        #name = info[0].findAll('a')
        #print name
        game_log_adv = game_log_adv.append({
                                    'ts_pct': (tds[1].text),
                                    'efg_pct': (tds[2].text),
                                    'fg3a_per_fga_pct': (tds[3].text),
                                    'fta_per_fga_pct': (tds[4].text),
                                    'orb_pct': float(tds[5].text),
                                    'drb_pct': float(tds[6].text),
                                    'trb_pct': float(tds[7].text),
                                    'ast_pct': float(tds[8].text),
                                    'stl_pct': float(tds[9].text),
                                    'blk_pct': float(tds[10].text),
                                    'tov_pct': (tds[11].text),
                                    'usg_pct': float(tds[12].text),
                                    'off_rtg': float(tds[13].text),
                                    'def_rtg': float(tds[14].text)}, 
                                ignore_index=True)
        
dat = pd.concat([game_log_basic, game_log_adv], axis=1, join_axes=[game_log_basic.index])

4
41
20
20
20
20
20
41
20
20
20
20
20
20
4
33
16
16
16
16
16
33
16
16
16
16
16
16


In [272]:
dat

,ast,blk,drb,fg,fg3,fg3_pct,fg3a,fg_pct,fga,ft,...,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,off_rtg,orb_pct,stl_pct,tov_pct,trb_pct,ts_pct,usg_pct
0,3.0,0.0,4.0,6.0,1.0,.250,4.0,.286,21.0,2.0,...,.310,.190,.143,70.0,0.0,1.4,8.2,6.0,.336,29.1
1,2.0,1.0,5.0,5.0,0.0,.000,2.0,.357,14.0,5.0,...,.357,.143,.500,88.0,3.2,0.0,10.5,9.2,.439,23.5
2,5.0,0.0,3.0,4.0,1.0,.333,3.0,.571,7.0,2.0,...,.643,.429,.429,105.0,3.9,3.6,26.5,7.6,.661,17.3
3,0.0,0.0,3.0,0.0,0.0,,0.0,.000,6.0,0.0,...,.000,.000,.000,0.0,0.0,0.0,0.0,6.6,.000,10.6
4,0.0,2.0,4.0,1.0,0.0,,0.0,.333,3.0,0.0,...,.333,.000,.000,54.0,13.4,0.0,40.0,19.5,.333,13.0
5,0.0,1.0,2.0,6.0,0.0,.000,1.0,.545,11.0,2.0,...,.545,.091,.182,100.0,4.1,0.0,14.4,5.9,.589,22.0
6,2.0,5.0,6.0,3.0,0.0,,0.0,.300,10.0,2.0,...,.300,.000,.200,73.0,12.7,1.9,21.6,18.4,.368,22.8
7,3.0,1.0,0.0,0.0,0.0,.000,1.0,.000,2.0,0.0,...,.000,.500,.000,71.0,7.6,0.0,0.0,3.7,.000,5.9
8,1.0,0.0,0.0,2.0,0.0,,0.0,.500,4.0,0.0,...,.500,.000,.000,114.0,11.4,0.0,0.0,5.5,.500,17.7
9,0.0,1.0,4.0,4.0,0.0,,0.0,1.000,4.0,1.0,...,.000,.500,.000,47.0,0.0,0.0,33.3,0.0,.000,16.6


## Larger scraping functions

In [10]:
def scrape_basic(table_basic):
    rows = table_basic.findAll('tr')
    rows = rows[:-1]
    game_log_basic = pd.DataFrame()
    for row in rows:
        if (len(row) == 21) | (len(row) == 20):
            tds = row.findAll('td')
            #print row.findAll('a')
            player = row.findAll('a')[0].text
            info = row.findAll('th')
            name = info[0].findAll('a')
            player_link = name[0].get('href')
            player_id = (player_link.split('/')[3])[:-5]
            game_log_basic = game_log_basic.append({'player': player,
                                        'link': player_link,
                                        'id': player_id,
                                        'mp': tds[0].text,
                                        'sec': int(time_to_sec(tds[0].text)),
                                        'fg': int(tds[1].text),
                                        'fga': int(tds[2].text),
                                        'fg_pct': tds[3].text,
                                        'fg3': int(tds[4].text),
                                        'fg3a': int(tds[5].text),
                                        'fg3_pct': tds[6].text,
                                        'ft': int(tds[7].text),
                                        'fta': int(tds[8].text),
                                        'ft_pct': tds[9].text,
                                        'orb': int(tds[10].text),
                                        'drb': int(tds[11].text),
                                        'trb': int(tds[12].text),
                                        'ast': int(tds[13].text),
                                        'stl': int(tds[14].text),
                                        'blk': int(tds[15].text),
                                        'tov': int(tds[16].text),
                                        'pf': int(tds[17].text),
                                        'pts': int(tds[18].text)},
                                        #'PM': int(tds[19].text)}, 
                                    ignore_index=True)
    return game_log_basic

In [11]:
def scrape_advanced(table_adv):
    rows = table_adv.findAll('tr')
    rows = rows[:-1]
    game_log_adv = pd.DataFrame()
    for row in rows:
        if len(row) == 16:
            tds = row.findAll('td')
            #print row.findAll('a')
            game_log_adv = game_log_adv.append({
                                        'ts_pct': (tds[1].text),
                                        'efg_pct': (tds[2].text),
                                        'fg3a_per_fga_pct': (tds[3].text),
                                        'fta_per_fga_pct': (tds[4].text),
                                        'orb_pct': float(tds[5].text),
                                        'drb_pct': float(tds[6].text),
                                        'trb_pct': float(tds[7].text),
                                        'ast_pct': float(tds[8].text),
                                        'stl_pct': float(tds[9].text),
                                        'blk_pct': float(tds[10].text),
                                        'tov_pct': (tds[11].text),
                                        'usg_pct': float(tds[12].text),
                                        'off_rtg': float(tds[13].text),
                                        'def_rtg': float(tds[14].text)}, 
                                    ignore_index=True)
    return game_log_adv

In [12]:
def join(game_log_basic, game_log_adv, g_id, g_num, team, date):
    dat = pd.concat([game_log_basic, game_log_adv], axis=1, join_axes=[game_log_basic.index])
    dat['g_num'] = np.full(dat.shape[0], g_num)
    dat['g_id'] = np.full(dat.shape[0], g_id)
    dat['team'] = np.full(dat.shape[0], team)
    dat['date'] = np.full(dat.shape[0], date)
    return dat

In [29]:
gid_error_lst = []

In [30]:
def scrape_box_score(link):
    gid = link.split('/')[4][:-5]
    print gid
    date = get_date(gid)
    page = urllib.urlopen(link).read()
    soup = BeautifulSoup(page, "lxml")
    g_num1, g_num2 = get_game_num(soup)
    team1, team2 = get_teams(soup)
    try:
        # Four tables -- home basic, home adv, away basic, away adv
        tables = soup.findAll('table', id=re.compile("(advanced|basic)"))
        d1 = join(scrape_basic(tables[0]), scrape_advanced(tables[1]), gid, g_num1, team1, date)
        d2 = join(scrape_basic(tables[2]), scrape_advanced(tables[3]), gid, g_num2, team2, date)
        frames = [d1, d2]
        result = pd.concat(frames)
        return result
    except ValueError:
        gid_error_lst.append(gid)

In [217]:
re.compile("(a|div)")

re.compile(r'(a|div)')

In [218]:
tables = soup.findAll('table', id=re.compile("(advanced|basic)"))

In [302]:
link = 'https://www.basketball-reference.com/boxscores/198401120HOU.html'
gid = link.split('/')[4][:-5]
print gid
date = get_date(gid)
page = urllib.urlopen(link).read()
soup = BeautifulSoup(page, "lxml")
g_num1, g_num2 = get_game_num(soup)
team1, team2 = get_teams(soup)
# Four tables -- home basic, home adv, away basic, away adv
tables = soup.findAll('table', id=re.compile("(advanced|basic)"))
d1 = join(scrape_basic(tables[0]), scrape_advanced(tables[1]), gid, g_num1, team1, date)
d2 = join(scrape_basic(tables[2]), scrape_advanced(tables[3]), gid, g_num2, team2, date)
frames = [d1, d2]
result = pd.concat(frames)

198401120HOU


In [307]:
result['date']

0     1984-01-12
1     1984-01-12
2     1984-01-12
3     1984-01-12
4     1984-01-12
5     1984-01-12
6     1984-01-12
7     1984-01-12
8     1984-01-12
0     1984-01-12
1     1984-01-12
2     1984-01-12
3     1984-01-12
4     1984-01-12
5     1984-01-12
6     1984-01-12
7     1984-01-12
8     1984-01-12
9     1984-01-12
10    1984-01-12
Name: date, dtype: object

In [304]:
result['player']

0          Kenny Carr
1         Calvin Natt
2           Fat Lever
3     Mychal Thompson
4          Jim Paxson
5       Clyde Drexler
6        Wayne Cooper
7        Eddie Jordan
8        Audie Norris
0         Robert Reid
1       Ralph Sampson
2           Phil Ford
3         Lewis Lloyd
4      Caldwell Jones
5       Allen Leavell
6        James Bailey
7       Rodney McCray
8         Elvin Hayes
9        Terry Teagle
10       Wally Walker
Name: player, dtype: object

In [422]:
link = 'https://www.basketball-reference.com/boxscores/201505120CLE.html'
box_df = scrape_box_score(link)

201505120CLE


In [424]:
box_df

,ast,blk,drb,fg,fg3,fg3_pct,fg3a,fg_pct,fga,ft,...,orb_pct,stl_pct,tov_pct,trb_pct,ts_pct,usg_pct,g_num,g_id,team,date
0,7.0,1.0,7.0,7.0,0.0,.000,4.0,.292,24.0,2.0,...,5.4,3.8,3.8,12.5,.316,28.0,5,201505120CLE,CHI,2015-05-12
1,3.0,1.0,6.0,9.0,3.0,.375,8.0,.500,18.0,8.0,...,8.3,0.0,4.4,13.0,.674,24.7,5,201505120CLE,CHI,2015-05-12
2,2.0,1.0,6.0,4.0,0.0,,0.0,.667,6.0,2.0,...,8.4,1.3,36.8,13.0,.727,12.0,5,201505120CLE,CHI,2015-05-12
3,4.0,0.0,4.0,5.0,2.0,.500,4.0,.625,8.0,7.0,...,0.0,0.0,0.0,6.4,.857,13.6,5,201505120CLE,CHI,2015-05-12
4,0.0,1.0,3.0,4.0,0.0,,0.0,.333,12.0,2.0,...,8.8,0.0,7.2,11.4,.388,24.2,5,201505120CLE,CHI,2015-05-12
5,3.0,0.0,0.0,1.0,1.0,.500,2.0,.250,4.0,0.0,...,0.0,2.4,0.0,0.0,.375,8.0,5,201505120CLE,CHI,2015-05-12
6,1.0,1.0,0.0,2.0,1.0,.333,3.0,.286,7.0,5.0,...,12.4,0.0,0.0,6.4,.519,23.6,5,201505120CLE,CHI,2015-05-12
7,0.0,0.0,2.0,2.0,0.0,.000,3.0,.333,6.0,0.0,...,0.0,0.0,14.3,16.5,.333,44.1,5,201505120CLE,CHI,2015-05-12
8,0.0,0.0,0.0,0.0,0.0,.000,1.0,.000,1.0,0.0,...,0.0,0.0,0.0,0.0,.000,9.5,5,201505120CLE,CHI,2015-05-12
0,3.0,3.0,3.0,6.0,1.0,.200,5.0,.600,10.0,0.0,...,11.8,0.0,0.0,9.9,.650,11.4,5,201505120CLE,CLE,2015-05-12


In [245]:
box_df[box_df['id'] == 'covinro01']

KeyError: 'id'

In [306]:
box_df

,ast,blk,drb,fg,fg3,fg3_pct,fg3a,fg_pct,fga,ft,...,off_rtg,orb_pct,stl_pct,tov_pct,trb_pct,ts_pct,usg_pct,g_num,g_id,team
0,7.0,1.0,7.0,7.0,0.0,.000,4.0,.292,24.0,2.0,...,85.0,5.4,3.8,3.8,12.5,.316,28.0,5,201505120CLE,CHI
1,3.0,1.0,6.0,9.0,3.0,.375,8.0,.500,18.0,8.0,...,144.0,8.3,0.0,4.4,13.0,.674,24.7,5,201505120CLE,CHI
2,2.0,1.0,6.0,4.0,0.0,,0.0,.667,6.0,2.0,...,102.0,8.4,1.3,36.8,13.0,.727,12.0,5,201505120CLE,CHI
3,4.0,0.0,4.0,5.0,2.0,.500,4.0,.625,8.0,7.0,...,186.0,0.0,0.0,0.0,6.4,.857,13.6,5,201505120CLE,CHI
4,0.0,1.0,3.0,4.0,0.0,,0.0,.333,12.0,2.0,...,87.0,8.8,0.0,7.2,11.4,.388,24.2,5,201505120CLE,CHI
5,3.0,0.0,0.0,1.0,1.0,.500,2.0,.250,4.0,0.0,...,109.0,0.0,2.4,0.0,0.0,.375,8.0,5,201505120CLE,CHI
6,1.0,1.0,0.0,2.0,1.0,.333,3.0,.286,7.0,5.0,...,129.0,12.4,0.0,0.0,6.4,.519,23.6,5,201505120CLE,CHI
7,0.0,0.0,2.0,2.0,0.0,.000,3.0,.333,6.0,0.0,...,59.0,0.0,0.0,14.3,16.5,.333,44.1,5,201505120CLE,CHI
8,0.0,0.0,0.0,0.0,0.0,.000,1.0,.000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,.000,9.5,5,201505120CLE,CHI
0,3.0,3.0,3.0,6.0,1.0,.200,5.0,.600,10.0,0.0,...,154.0,11.8,0.0,0.0,9.9,.650,11.4,5,201505120CLE,CLE


## Collecting Box Score Links

In [ ]:
#https://www.basketball-reference.com/leagues/NBA_2017_games.html

In [348]:
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
yrs = range(1984,2018)

In [349]:
sched_links = ['https://www.basketball-reference.com/leagues/NBA_' + str(yr) + '_games-' + month + 
               '.html' for yr in yrs for month in months]

In [350]:
len(sched_links)

306

In [351]:
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1988_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1989_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1990_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1991_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1992_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1993_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1994_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1995_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1996_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1997_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1999_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1999_games-november.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1999_games-december.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_1999_games-january.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_2000_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_2005_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_2006_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_2012_games-october.html')
sched_links.remove('https://www.basketball-reference.com/leagues/NBA_2012_games-november.html')

In [352]:
len(sched_links)

287

In [353]:
def get_month_links(link):
    page = urllib.urlopen(link).read()
    soup = BeautifulSoup(page, "lxml")
    sched = soup.find('table', id='schedule')
    links = sched.findAll('a')
    valid_links = ['https://www.basketball-reference.com' + link.get('href') for i, link in enumerate(links) if i%4==3]
    return valid_links

In [354]:
link1 = 'https://www.basketball-reference.com/leagues/NBA_1984_games-january.html'
link2 = 'https://www.basketball-reference.com/leagues/NBA_1984_games-february.html'
lst = []

In [332]:
lst += get_month_links(link1)
lst += get_month_links(link2)

In [330]:
len(get_month_links(link2))

173

In [333]:
len(lst)

326

In [65]:

link = 'https://www.basketball-reference.com/leagues/NBA_2017_games-october.html'
page = urllib.urlopen(link).read()
soup = BeautifulSoup(page, "lxml")

In [66]:
sched = soup.find('table', id='schedule')

In [175]:
links = sched.findAll('a')

In [179]:
['https://www.basketball-reference.com' + link.get('href') for i, link in enumerate(links) if i%4==3]

['https://www.basketball-reference.com/boxscores/201610250CLE.html',
 'https://www.basketball-reference.com/boxscores/201610250GSW.html',
 'https://www.basketball-reference.com/boxscores/201610250POR.html',
 'https://www.basketball-reference.com/boxscores/201610260BOS.html',
 'https://www.basketball-reference.com/boxscores/201610260IND.html',
 'https://www.basketball-reference.com/boxscores/201610260LAL.html',
 'https://www.basketball-reference.com/boxscores/201610260MEM.html',
 'https://www.basketball-reference.com/boxscores/201610260MIL.html',
 'https://www.basketball-reference.com/boxscores/201610260NOP.html',
 'https://www.basketball-reference.com/boxscores/201610260ORL.html',
 'https://www.basketball-reference.com/boxscores/201610260PHI.html',
 'https://www.basketball-reference.com/boxscores/201610260PHO.html',
 'https://www.basketball-reference.com/boxscores/201610260TOR.html',
 'https://www.basketball-reference.com/boxscores/201610270ATL.html',
 'https://www.basketball-reference

In [191]:
link = 'https://www.basketball-reference.com/leagues/NBA_2017_games-april.html'
page = urllib.urlopen(link).read()
soup = BeautifulSoup(page, "lxml")

In [192]:
sched = soup.find('table', id='schedule')

In [193]:
links = sched.findAll('a')

In [194]:
links

[<a href="/boxscores/index.fcgi?month=4&amp;day=1&amp;year=2017">Sat, Apr 1, 2017</a>,
 <a href="/teams/ORL/2017.html">Orlando Magic</a>,
 <a href="/teams/BRK/2017.html">Brooklyn Nets</a>,
 <a href="/boxscores/201704010BRK.html">Box Score</a>,
 <a href="/boxscores/index.fcgi?month=4&amp;day=1&amp;year=2017">Sat, Apr 1, 2017</a>,
 <a href="/teams/ATL/2017.html">Atlanta Hawks</a>,
 <a href="/teams/CHI/2017.html">Chicago Bulls</a>,
 <a href="/boxscores/201704010CHI.html">Box Score</a>,
 <a href="/boxscores/index.fcgi?month=4&amp;day=1&amp;year=2017">Sat, Apr 1, 2017</a>,
 <a href="/teams/LAL/2017.html">Los Angeles Lakers</a>,
 <a href="/teams/LAC/2017.html">Los Angeles Clippers</a>,
 <a href="/boxscores/201704010LAC.html">Box Score</a>,
 <a href="/boxscores/index.fcgi?month=4&amp;day=1&amp;year=2017">Sat, Apr 1, 2017</a>,
 <a href="/teams/SAC/2017.html">Sacramento Kings</a>,
 <a href="/teams/MIN/2017.html">Minnesota Timberwolves</a>,
 <a href="/boxscores/201704010MIN.html">Box Score</a>,


In [355]:
lst = []
for link in sched_links:
    print link
    lst += get_month_links(link)

https://www.basketball-reference.com/leagues/NBA_1984_games-october.html
https://www.basketball-reference.com/leagues/NBA_1984_games-november.html
https://www.basketball-reference.com/leagues/NBA_1984_games-december.html
https://www.basketball-reference.com/leagues/NBA_1984_games-january.html
https://www.basketball-reference.com/leagues/NBA_1984_games-february.html
https://www.basketball-reference.com/leagues/NBA_1984_games-march.html
https://www.basketball-reference.com/leagues/NBA_1984_games-april.html
https://www.basketball-reference.com/leagues/NBA_1984_games-may.html
https://www.basketball-reference.com/leagues/NBA_1984_games-june.html
https://www.basketball-reference.com/leagues/NBA_1985_games-october.html
https://www.basketball-reference.com/leagues/NBA_1985_games-november.html
https://www.basketball-reference.com/leagues/NBA_1985_games-december.html
https://www.basketball-reference.com/leagues/NBA_1985_games-january.html
https://www.basketball-reference.com/leagues/NBA_1985_gam

In [356]:
len(lst)

40989

In [359]:
link_df = pd.DataFrame(data = {'links':np.array(lst)})

In [361]:
link_df

,links
0,https://www.basketball-reference.com/boxscores...
1,https://www.basketball-reference.com/boxscores...
2,https://www.basketball-reference.com/boxscores...
3,https://www.basketball-reference.com/boxscores...
4,https://www.basketball-reference.com/boxscores...
5,https://www.basketball-reference.com/boxscores...
6,https://www.basketball-reference.com/boxscores...
7,https://www.basketball-reference.com/boxscores...
8,https://www.basketball-reference.com/boxscores...
9,https://www.basketball-reference.com/boxscores...


In [362]:
link_df.to_excel('box_score_link.xlsx')

In [ ]:
lst[5081]

In [402]:
lst2 = lst[:]

In [15]:
link_df = pd.read_excel('data/box_score_link.xlsx')

In [16]:
lst = link_df['links']

In [199]:
str(lst[19082])

'https://www.basketball-reference.com/boxscores/200010310ATL.html'

In [389]:
str(lst[39680])

'https://www.basketball-reference.com/boxscores/201610250CLE.html'

In [393]:
len(lst)

40989

In [395]:
#box_score_data = pd.DataFrame()
df_list = []
# Removing first two seasons
for link in lst[39680:]:
    df_list.append(scrape_box_score(str(link)))
    #box_score_data = box_score_data.append(scrape_box_score(link))

201610250CLE
201610250GSW
201610250POR
201610260BOS
201610260IND
201610260LAL
201610260MEM
201610260MIL
201610260NOP
201610260ORL
201610260PHI
201610260PHO
201610260TOR
201610270ATL
201610270CHI
201610270POR
201610270SAC
201610280BRK
201610280DAL
201610280DET
201610280MIA
201610280NOP
201610280OKC
201610280TOR
201610280UTA
201610290CHI
201610290CHO
201610290CLE
201610290DEN
201610290MIL
201610290NYK
201610290PHI
201610290SAC
201610290SAS
201610300DET
201610300HOU
201610300LAC
201610300MEM
201610300MIA
201610300OKC
201610300PHO
201610310ATL
201610310BRK
201610310LAC
201610310TOR
201611010CLE
201611010DET
201611010IND
201611010MIA
201611010MIN
201611010NOP
201611010PHI
201611010POR
201611010SAS
201611020ATL
201611020BOS
201611020BRK
201611020CHO
201611020LAC
201611020MEM
201611020NYK
201611020PHO
201611020UTA
201611020WAS
201611030CLE
201611030GSW
201611030MIL
201611030MIN
201611030ORL
201611040BRK
201611040CHI
201611040DAL
201611040LAL
201611040MEM
201611040NOP
201611040TOR
201611040UTA

In [396]:
df = pd.concat(df_list)

In [397]:
df.to_excel('data/box_scores/box_score_2017.xlsx')

# Working with invalid links

In [400]:
# bad links
gid_error_lst

['198902050BOS',
 '198902200SAS',
 '199205270CHI',
 '199303270ORL',
 '199412060SEA',
 '199601270WSB',
 '199612180GSW',
 '199702250WSB',
 '199802190LAC',
 '199804290SAS',
 '200211080WAS',
 '200211150LAL',
 '200212030GSW',
 '200212070DEN',
 '200212160PHO',
 '200301060BOS',
 '200301200LAL',
 '200303040NYK',
 '200404060ATL',
 '200405240IND',
 '200501240LAC',
 '200505090DET',
 '200511150PHI',
 '200512110MIA',
 '200601290TOR',
 '200603120MIA',
 '200603130LAC',
 '200604070NYK',
 '200604220CLE',
 '200604270DEN',
 '200704100WAS',
 '200712200DEN',
 '200804270WAS',
 '200901020TOR',
 '200904130DAL',
 '200905110DAL',
 '200911040HOU',
 '200911160ORL',
 '200912120WAS',
 '201002180CLE',
 '201004250SAS',
 '201012010ATL',
 '201102230SAS',
 '201103060DAL',
 '201104190DAL',
 '201012010ATL',
 '201102230SAS',
 '201103060DAL',
 '201104190DAL',
 '201201290MIA',
 '201202170MEM',
 '201202190MIN',
 '201211280WAS',
 '201212020LAL',
 '201301060LAL',
 '201303010DEN',
 '201312020SAS',
 '201401130NYK',
 '201402080PHO

In [446]:
def scrape_basic_2(table_basic):
    rows = table_basic.findAll('tr')
    rows = rows[:-1]
    game_log_basic = pd.DataFrame()
    for row in rows:
        if (len(row) == 21) | (len(row) == 20):
            tds = row.findAll('td')
            #print row.findAll('a')
            player = row.findAll('a')[0].text
            info = row.findAll('th')
            name = info[0].findAll('a')
            player_link = name[0].get('href')
            player_id = (player_link.split('/')[3])[:-5]
            game_log_basic = game_log_basic.append({'player': player,
                                        'link': player_link,
                                        'id': player_id,
                                        'mp': tds[0].text,
                                        'sec': (time_to_sec(tds[0].text)),
                                        'fg': (tds[1].text),
                                        'fga': (tds[2].text),
                                        'fg_pct': tds[3].text,
                                        'fg3': (tds[4].text),
                                        'fg3a': (tds[5].text),
                                        'fg3_pct': tds[6].text,
                                        'ft': (tds[7].text),
                                        'fta': (tds[8].text),
                                        'ft_pct': tds[9].text,
                                        'orb': (tds[10].text),
                                        'drb': (tds[11].text),
                                        'trb': (tds[12].text),
                                        'ast': (tds[13].text),
                                        'stl': (tds[14].text),
                                        'blk': (tds[15].text),
                                        'tov': (tds[16].text),
                                        'pf': (tds[17].text),
                                        'pts': (tds[18].text)},
                                        #'PM': int(tds[19].text)}, 
                                    ignore_index=True)
    return game_log_basic

In [447]:
def scrape_advanced_2(table_adv):
    rows = table_adv.findAll('tr')
    rows = rows[:-1]
    game_log_adv = pd.DataFrame()
    for row in rows:
        if len(row) == 16:
            tds = row.findAll('td')
            #print row.findAll('a')
            game_log_adv = game_log_adv.append({
                                        'ts_pct': (tds[1].text),
                                        'efg_pct': (tds[2].text),
                                        'fg3a_per_fga_pct': (tds[3].text),
                                        'fta_per_fga_pct': (tds[4].text),
                                        'orb_pct': (tds[5].text),
                                        'drb_pct': (tds[6].text),
                                        'trb_pct': (tds[7].text),
                                        'ast_pct': (tds[8].text),
                                        'stl_pct': (tds[9].text),
                                        'blk_pct': (tds[10].text),
                                        'tov_pct': (tds[11].text),
                                        'usg_pct': (tds[12].text),
                                        'off_rtg': (tds[13].text),
                                        'def_rtg': (tds[14].text)}, 
                                    ignore_index=True)
    return game_log_adv

In [448]:
def join_2(game_log_basic, game_log_adv, g_id, g_num, team, date):
    dat = pd.concat([game_log_basic, game_log_adv], axis=1, join_axes=[game_log_basic.index])
    dat['g_num'] = np.full(dat.shape[0], g_num)
    dat['g_id'] = np.full(dat.shape[0], g_id)
    dat['team'] = np.full(dat.shape[0], team)
    dat['date'] = np.full(dat.shape[0], date)
    return dat

In [449]:
def scrape_box_score_2(link):
    gid = link.split('/')[4][:-5]
    print gid
    date = get_date(gid)
    page = urllib.urlopen(link).read()
    soup = BeautifulSoup(page, "lxml")
    g_num1, g_num2 = get_game_num(soup)
    team1, team2 = get_teams(soup)
    #try:
    # Four tables -- home basic, home adv, away basic, away adv
    tables = soup.findAll('table', id=re.compile("(advanced|basic)"))
    d1 = join_2(scrape_basic_2(tables[0]), scrape_advanced_2(tables[1]), gid, g_num1, team1, date)
    d2 = join_2(scrape_basic_2(tables[2]), scrape_advanced_2(tables[3]), gid, g_num2, team2, date)
    frames = [d1, d2]
    result = pd.concat(frames)
    return result
    #except ValueError:
    #    gid_error_lst.append(gid)

In [450]:
time_to_sec([])

AttributeError: 'list' object has no attribute 'split'

In [451]:
link = 'https://www.basketball-reference.com/boxscores/201704100PHI.html'
box_df = scrape_box_score_2(link)

201704100PHI


In [452]:
box_df

,ast,blk,drb,fg,fg3,fg3_pct,fg3a,fg_pct,fga,ft,...,orb_pct,stl_pct,tov_pct,trb_pct,ts_pct,usg_pct,g_num,g_id,team,date
0,5,1,1,11,3,.375,8,.550,20,2,...,3.6,2.8,4.6,3.4,.647,26.6,81,201704100PHI,IND,2017-04-10
1,3,0,4,10,0,,0,.667,15,0,...,15.5,1.5,6.3,14.9,.667,21.2,81,201704100PHI,IND,2017-04-10
2,6,0,2,4,0,.000,2,.500,8,7,...,0.0,1.5,20.7,3.7,.651,19.4,81,201704100PHI,IND,2017-04-10
3,1,3,10,6,0,,0,.750,8,6,...,13.7,5.4,27.3,28.6,.846,22.9,81,201704100PHI,IND,2017-04-10
4,5,0,3,1,0,,0,.333,3,0,...,0.0,1.9,40.0,7.1,.333,8.4,81,201704100PHI,IND,2017-04-10
5,0,0,4,4,1,.200,5,.400,10,0,...,0.0,0.0,9.1,11.7,.450,23.0,81,201704100PHI,IND,2017-04-10
6,5,0,2,3,0,,0,.429,7,0,...,0.0,0.0,0.0,6.0,.403,15.7,81,201704100PHI,IND,2017-04-10
7,0,0,4,8,0,,0,.800,10,1,...,13.4,2.6,8.4,19.3,.781,27.2,81,201704100PHI,IND,2017-04-10
8,2,0,0,1,1,1.000,1,.500,2,0,...,0.0,0.0,33.3,0.0,.750,10.8,81,201704100PHI,IND,2017-04-10
9,3,1,0,1,0,,0,.500,2,0,...,63.2,0.0,0.0,30.3,.500,10.8,81,201704100PHI,IND,2017-04-10


In [454]:
invalid_links = ['https://www.basketball-reference.com/boxscores/' + invalid_gid + '.html'
                 for invalid_gid in gid_error_lst]

In [455]:
#box_score_data = pd.DataFrame()
df_list = []
# Removing first two seasons
for link in invalid_links:
    df_list.append(scrape_box_score_2(str(link)))

198902050BOS
198902200SAS
199205270CHI
199303270ORL
199412060SEA
199601270WSB
199612180GSW
199702250WSB
199802190LAC
199804290SAS
200211080WAS
200211150LAL
200212030GSW
200212070DEN
200212160PHO
200301060BOS
200301200LAL
200303040NYK
200404060ATL
200405240IND
200501240LAC
200505090DET
200511150PHI
200512110MIA
200601290TOR
200603120MIA
200603130LAC
200604070NYK
200604220CLE
200604270DEN
200704100WAS
200712200DEN
200804270WAS
200901020TOR
200904130DAL
200905110DAL
200911040HOU
200911160ORL
200912120WAS
201002180CLE
201004250SAS
201012010ATL
201102230SAS
201103060DAL
201104190DAL
201012010ATL
201102230SAS
201103060DAL
201104190DAL
201201290MIA
201202170MEM
201202190MIN
201211280WAS
201212020LAL
201301060LAL
201303010DEN
201312020SAS
201401130NYK
201402080PHO
201402120GSW
201403140BOS
201404010DAL
201404190LAC
201404210OKC
201411130MEM
201501060SAS
201505200ATL
201511110DAL
201511190LAC
201512060MEM
201512070CHI
201512130TOR
201512230ATL
201602060MEM
201603040DEN
201603140UTA
201605080OKC

In [456]:
df = pd.concat(df_list)

In [457]:
df.to_excel('data/box_scores/box_score_invalid.xlsx')